In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Relax Data Challenge

## Problem:
Identify which factors predict future user adoption
+ Adopted user: user who has logged into the product on three separate days in at least one seven-day period

## Data

The first step is to import the data into Python to do some initial data exploration to figure out if the data has any missing values or irregularities.

### Users

In [7]:
# Import user information
users = pd.read_csv('takehome_users.csv', 
                    index_col = 'object_id', 
                    parse_dates = [1], 
                    encoding = 'iso-8859-1')

users.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [9]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 9 columns):
creation_time                 12000 non-null datetime64[ns]
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 937.5+ KB


From the initial findings, it appears that the data has some missing values under ``last_session_creation_time`` and ``invited_by_user_id``. Also, according to the data documentation, the column ``last_session_creation_time`` is encoded as a unix timestamp which may make more sense if it's converted from type float64 to a timestamp.

### User Engagement

In [8]:
# Import user engagement
user_engage = pd.read_csv('takehome_user_engagement.csv',
                          index_col = 'time_stamp',
                         parse_dates = True)

user_engage.head()

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1
